In [ ]:
!pip install tensorflow==2.8.2

In [ ]:
#Import data manipulation libraries
import numpy as np
import pandas as pd
from tqdm import tqdm

#Importing ML/DL libraries
from sklearn.metrics import roc_curve, auc,precision_recall_fscore_support, auc
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.layers import Dropout

df = pd.read_csv('yahoo3.csv')
dimension=len(df.columns)

def get_generator(optimizer):
    generator = Sequential()
    generator.add(Dense(64, input_dim=dimension))
    generator.add(Activation('tanh'))

    generator.add(Dense(128))
    generator.add(Activation('tanh'))

    generator.add(Dense(256))
    generator.add(Activation('tanh'))

    generator.add(Dense(256))
    generator.add(Activation('tanh'))

    generator.add(Dense(512))
    generator.add(Activation('tanh'))

    generator.add(Dense(dimension, activation='tanh'))

    generator.compile(loss='binary_crossentropy', optimizer=optimizer)

    return generator


def get_discriminator(optimizer):
    discriminator = Sequential()
    discriminator.add(Dense(256, input_dim=dimension))
    discriminator.add(Activation('relu'))
    discriminator.add(Dropout(0.2))

    discriminator.add(Dense(128))
    discriminator.add(Activation('relu'))
    discriminator.add(Dropout(0.2))

    discriminator.add(Dense(128))
    discriminator.add(Activation('relu'))
    discriminator.add(Dropout(0.2))

    discriminator.add(Dense(128))
    discriminator.add(Activation('relu'))
    discriminator.add(Dropout(0.2))

    discriminator.add(Dense(128))
    discriminator.add(Activation('relu'))
    discriminator.add(Dropout(0.2))

    discriminator.add(Dense(1))
    discriminator.add(Activation('sigmoid'))

    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)

    return discriminator


def get_gan_network(discriminator, generator, optimizer, input_dim=dimension):
    discriminator.trainable = False
    gan_input = Input(shape=(input_dim,))
    x = generator(gan_input)
    gan_output = discriminator(x)

    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizer)

    return gan

if __name__ == '__main__':
    learning_rate = 0.00001
    batch_size = 150
    epochs = 50
    adam = Adam(lr=learning_rate, beta_1=0.5)
    actual = pd.read_csv('yahoo3.csv', usecols=['Class'])
    y_true = actual
    dataset = pd.read_csv('yahoo3.csv',usecols=lambda column: column != 'Class')
    tmp=len(dataset)
    dimension=len(dataset.columns)
    dataset = dataset.astype('float32')
    dataset = np.array(dataset)
    normal_data = dataset[actual['Class'] == 0]
    train_size = int(tmp * 0.7)
    test_size = tmp
    train_data=[]
    for i in range(train_size):
        train_data.append(normal_data[i])
    test_data=dataset
    train_data = np.array(train_data)
    batch_count = train_data.shape[0] // batch_size
    pbar = tqdm(total=epochs * batch_count)
    gan_loss = []
    discriminator_loss = []

    # Inititalizing the network
    generator = get_generator(adam)
    discriminator = get_discriminator(adam)
    gan = get_gan_network(discriminator, generator, adam, input_dim=dimension)

    # Print initial weights of the generator
    print("Initial Weights of Generator:")
    for layer in generator.layers:
        if hasattr(layer, 'get_weights'):
            weights = layer.get_weights()
            if weights:
                print(f"{layer.name} Weights: {weights}")

    # Print initial weights of the discriminator
    print("\nInitial Weights of Discriminator:")
    for layer in discriminator.layers:
        if hasattr(layer, 'get_weights'):
            weights = layer.get_weights()
            if weights:
                print(f"{layer.name} Weights: {weights}")


    for epoch in range(epochs):
        for index in range(batch_count):
            pbar.update(1)
            # Creating a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batch_size, dimension])

            # Generate fake samples
            generated_images = generator.predict_on_batch(noise)

            # Obtain a batch of normal network packets
            image_batch = train_data[index * batch_size: (index + 1) * batch_size]

            X = np.vstack((generated_images, image_batch))
            y_dis = np.ones(2 * batch_size)
            y_dis[:batch_size] = 0

            # Train discriminator
            discriminator.trainable = True
            d_loss = discriminator.train_on_batch(X, y_dis)

            # Train generator
            noise = np.random.uniform(0, 1, size=[batch_size, dimension])
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            g_loss = gan.train_on_batch(noise, y_gen)

            # Record the losses
            discriminator_loss.append(d_loss)
            gan_loss.append(g_loss)

    dataframe1 = pd.read_csv('yahoo3.csv', usecols=['Class'], engine='python')
    dataset1 =pd.read_csv('yahoo3.csv',usecols=lambda column: column != 'Class')
    test_x_predictions = discriminator.predict(dataset1)
    y_pred = np.array(test_x_predictions)
    per = np.percentile(test_x_predictions, 10)
    inds = (y_pred > per)

    inds_comp = (y_pred <= per)
    y_pred[inds] = 0
    y_pred[inds_comp] = 1
    precision_gan, recall_gan, f1_score_gan, _ = precision_recall_fscore_support(actual, y_pred, average='binary')

    # Calculate ROC curve
    fpr6, tpr6, _ = roc_curve(actual, y_pred)

    # Calculate AUC (Area Under the ROC Curve)
    auc_roc_gan = auc(fpr6, tpr6)

    print ('Precision :',precision_gan ),
    print ('Recall :',recall_gan ),
    print ('F1 :',f1_score_gan )
    print('AUC ROC: ',auc_roc_gan)